Importing necessary libraries

In [7]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#nltk.download('stopwords')
#nltk.download('punkt')

We first start by importing googlereviews5000.xlsx. We dropped less useful columns to make our dataset more compact.

In [8]:
df = pd.read_excel('googlereviews5000.xlsx')

df = df.drop(columns = ['rating', 'reviewUrl', 'reviewerNumberOfReviews', 'title', 'likesCount', 'isAdvertisement', 'reviewerId'])
df.rename(columns= {'stars' : 'rating',
                    'reviewContext/Reservation recommended' : 'reservation_recommended',
                    'reviewContext/Visited on' : 'visited_on',
                    'reviewContext/Wait time' : 'wait_time',
                    'publishedAtDate' : 'published_date',
                    'isLocalGuide' : 'is_local_guide',
                    'originalLanguage' : 'original_language',
                    'textTranslated' : 'text_translated'}, inplace=True)

print(df.columns)

print(df.isnull().sum()) 


Index(['rating', 'reservation_recommended', 'visited_on', 'wait_time',
       'published_date', 'is_local_guide', 'text', 'original_language',
       'text_translated'],
      dtype='object')
rating                        0
reservation_recommended    1988
visited_on                 1780
wait_time                  1899
published_date                0
is_local_guide                0
text                       2612
original_language          2607
text_translated            4293
dtype: int64


Let's see how many unique languages the reviews contain. We keep the original text for English reviews and the translated text for non-English reviews.

In [9]:
print(df['original_language'].unique())

df['review_text'] = df.apply(lambda row: row["text"] if row["original_language"] == "en" else row["text_translated"], axis = 1)
df = df.drop(columns = ["text", "text_translated"])

print(df.columns)
print(df.shape[0])


[nan 'en' 'id' 'th' 'ko' 'zh-Hant' 'ja' 'zh' 'pl' 'da' 'ru' 'iw' 'vi' 'lt'
 'fr' 'zu' 'de' 'cs' 'es' 'tr' 'ar' 'it' 'uk' 'ro' 'fil' 'hi' 'pt' 'my'
 'sv' 'no' 'la' 'ta' 'fy' 'el' 'ceb' 'ms' 'is' 'nl' 'mr' 'hu' 'gu']
Index(['rating', 'reservation_recommended', 'visited_on', 'wait_time',
       'published_date', 'is_local_guide', 'original_language', 'review_text'],
      dtype='object')
5000


We proceed to clean other aspects of the data. 
We first slice the data in the 'publishedAtDate' column to convert it to a date object.
We fill in empty values in 'Visited on' with the corresponding date of the review. Here we are assuming the reviewers published the review on the same day that they visited USS to maximize data retention while maintaining data integrity.

In [10]:
print(df.dtypes)

df['published_date'] = df['published_date'].str[:10]

df['published_date'] = pd.to_datetime(df['published_date'])

df['visited_on'] = df['published_date'].apply(lambda x: "Weekend" if pd.notna(x) and x.weekday() >= 5 else "Weekday")

print(df['published_date'][0])

rating                      int64
reservation_recommended    object
visited_on                 object
wait_time                  object
published_date             object
is_local_guide               bool
original_language          object
review_text                object
dtype: object
2025-02-27 00:00:00


We notice there are a large number of missing data in the 'Cleaned Text' column, indicating many reviewers did not write any text as part of the review. However we could still keep these reviews as they still contain relevant information on the time of the review and corresponding rating. We will store these textless data in a separate dataframe.

In [11]:
print(df.isnull().sum()) 

df_no_text = df[df["review_text"].isna() | (df["review_text"].str.strip() == "")]
df_text = df[~df["review_text"].isna() & (df["review_text"].str.strip() != "")]

rating                        0
reservation_recommended    1988
visited_on                    0
wait_time                  1899
published_date                0
is_local_guide                0
original_language          2607
review_text                2620
dtype: int64


We save these 2 dataframe into 2 separate excel file to do further analysis.

In [12]:
df_no_text.to_excel("google reviews cleaned df_no_text.xlsx", index = False)
df_text.to_excel("google reviews cleaned df_text.xlsx", index = False)